# 뉴스 헤드라인 분류하기 (SageMaker 버전)

> 이 노트북은 세이지메이커 스튜디오의 `Python 3 (PyTorch 1.13 Python 3.9 CPU Optimized)` 커널에서 잘 작동합니다.

이 예제에서는 사용자 정의 스크립트와 [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) 프레임워크를 사용하여 뉴스 헤드라인 분류 모델을 훈련합니다.

이 "SageMaker" 노트북에서는 Amazon SageMaker 훈련 작업에서 모델을 훈련하고, 매니지드 실시간 추론 엔드포인트로 배포하는 방법을 보여드립니다.

> ⚠️ 여기서는 여기 노트북 자체에서 훈련 및 추론을 실행하는 방법을 보여주는, 동반된 ["Headline Classifier Local" notebook](Headline%20Classifier%20Local.ipynb)을 이미 실행한 것으로 가정합니다.

## 설치 및 설정

로컬 노트북에서와 마찬가지로, 시작하기 전에 위젯 라이브러리가 설정되어 있는지 확인하겠습니다.

🟢 하지만 **로컬 노트북과 달리** HF Transformers를 **설치할 필요가 없다**는 점에 유의하세요: 실제 훈련과 추론은 이 커널이 아닌 컨테이너화된 작업에서 이루어지기 때문입니다.

> ℹ️ (실제로 동일한 커널 이미지와 동일한 인스턴스에서 여러 개의 세이지메이커 스튜디오 노트북을 시작하면 환경을 공유하게 됩니다: 따라서 로컬 노트북을 실행하고 동일한 커널을 선택했다고 가정하면 모든 것이 이미 설치되어 있습니다).

In [ ]:
%pip install "ipywidgets<8" "sagemaker>2.140,<3"

설치가 완료되면 나머지 노트북에서 사용할 라이브러리와 Python 내장 기능을 로드합니다.

[%autoreload magic](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html)은 로컬 .py 파일로 작업할 때 유용합니다. 셀을 실행할 때마다 라이브러리를 다시 로드하면 노트북 커널을 재시작할 필요 없이 로컬에서 편집/업데이트된 스크립트를 사용할 수 있기 때문입니다.

🟢 이번에는 로컬 노트북에서는 필요 없던 **AWS libraries**를 사용하겠습니다:

- `boto3`, [general-purpose AWS SDK for Python](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)
- `sagemaker`,[high-level Python SDK for Amazon SageMaker](https://sagemaker.readthedocs.io/en/stable/)

이 두 라이브러리는 모두 오픈소스이며, PyPI와 GitHub에 게시되어 있습니다.

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import os  # Operating system utils e.g. file paths

# External Dependencies:
import boto3  # General AWS SDK for Python
import ipywidgets as widgets  # Interactive prediction widget
import pandas as pd  # Utilities for working with data tables (dataframes)
import sagemaker  # High-level Python SDK for Amazon SageMaker

local_dir = "data"

## 데이터셋 준비 및 업로드하기

이 예에서는 [Registry of Open Data on AWS](https://registry.opendata.aws/fast-ai-nlp/)의 퍼블릭 리포지토리에서 **FastAi AG News** 데이터셋을 다운로드합니다. 이 데이터셋에는 뉴스 헤드라인과 그에 해당하는 토픽 클래스의 테이블이 포함되어 있습니다.

In [ ]:
%%time
# Download the AG News data from the Registry of Open Data on AWS.
!mkdir -p {local_dir}
!aws s3 cp s3://fast-ai-nlp/ag_news_csv.tgz {local_dir} --no-sign-request

# Un-tar the AG News data.
!tar zxf {local_dir}/ag_news_csv.tgz -C {local_dir}/ --strip-components=1 --no-same-owner
print("Done!")

데이터를 다운로드하고 추출한 후 아래와 같이 몇 가지 예를 살펴볼 수 있습니다:

In [ ]:
column_names = ["CATEGORY", "TITLE", "CONTENT"]
# we use the train.csv only
df = pd.read_csv(f"{local_dir}/train.csv", names=column_names, header=None, delimiter=",")
# shuffle the DataFrame rows
df = df.sample(frac=1, random_state=1337)

# Make the (1-indexed) category classes more readable:
class_names = ["Other", "World", "Sports", "Business", "Sci/Tech"]
idx2label = {ix: name for ix, name in enumerate(class_names)}
label2idx = {name: ix for ix, name in enumerate(class_names)}

df = df.replace({"CATEGORY": idx2label})
df.head()

이번 연습에서는 **아래의 값만 사용하겠습니다**:

- 뉴스 기사의 **title** (Headline)을 인풋으로 사용합니다.
- 예측할 목표 변수로 **category**를 사용합니다.

이 데이터 세트에는 아래와 같이 4개의 균등하게 분포된 주제 클래스가 포함되어 있습니다.

> ℹ️ **'Other'는 어떻게 할까요?:** 원시 데이터 세트는 1~4 사이의 숫자로 범주를 나타내며, 우리의 모델은 0부터 시작하는 숫자를 예상하기 때문에, 데이터 준비를 단순하게 유지하고 클래스의 혼란스러운 추가 숫자 표현을 피하기 위해 사용하지 않는 'Other' 클래스를 삽입했습니다.

In [ ]:
df["CATEGORY"].value_counts()

지금까지 새로운 것은 없습니다...

🟢 세이지메이커 훈련의 주요 차이점은 [훈련 작업에서 액세스할 수 있는 곳에](https://docs.aws.amazon.com/sagemaker/latest/dg/model-access-training-data.html) 데이터셋을 **업로드**해야 한다는 것입니다.

여기서는 [SageMaker default bucket](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-ex-bucket.html)을 사용하여 데이터를 [Amazon S3](https://docs.aws.amazon.com/AmazonS3/latest/userguide/Welcome.html)에 업로드하겠습니다. 원하는 경우 버킷과 폴더 접두사를 사용자 지정할 수 있습니다. 훈련 데이터와 테스트 데이터를 같은 폴더에 두 파일만 저장하지 말고 별도의 S3 폴더로 분리하는 것이 좋습니다.

In [ ]:
bucket_name = sagemaker.Session().default_bucket()
s3_prefix = "sm101/news"

s3 = boto3.resource("s3")

s3.Bucket(bucket_name).upload_file(f"{local_dir}/train.csv", f"{s3_prefix}/train/train.csv")
train_s3_uri = f"s3://{bucket_name}/{s3_prefix}/train"
print(f"train_s3_uri: {train_s3_uri}")

s3.Bucket(bucket_name).upload_file(f"{local_dir}/test.csv", f"{s3_prefix}/test/test.csv")
test_s3_uri = f"s3://{bucket_name}/{s3_prefix}/test"
print(f"test_s3_uri: {test_s3_uri}")

## 훈련 매개변수 정의

[Hugging Face Hub](https://huggingface.co/models)에서 (비교적 작은) 사전 훈련된 모델을 미세 조정하고, 낮은 수준의 훈련 루프를 처음부터 작성하는 대신 높은 수준의 [Trainer API](https://huggingface.co/docs/transformers/main_classes/trainer)를 사용할 것입니다.

🟢 훈련 스크립트는 궁극적으로 이전과 비슷한 파라미터를 사용하지만 이번에는 **훈련 작업 API**를 통해 전달할 것입니다.

여기 노트북에서 **JSON 직렬화 가능 매개변수**를 정의한 다음, 이를 사용하여 나중에 `transformers.TrainingArguments`를 빌드할 것입니다:

In [ ]:
hyperparameters = {
    "model_id": "amazon/bort",  # ID of the pre-trained model to start from
    "class_names": ",".join(class_names),  # Comma-separated list of category names
    "num_train_epochs": 3,  # This time, we'll actually train for a full 3 epochs
    "per_device_train_batch_size": 32,  # Note this is higher than we could set on local hardware
    "per_device_eval_batch_size": 64,  # Note this is higher than we could set on local hardware
    "warmup_steps": 500,  # Higher than we could set with the reduced local training
}
hyperparameters

# 메트릭 정의

학습된 모델의 품질을 측정하는 방법을 정의하고 이 정보를 세이지메이커에 표시하여 메트릭 로깅, 자동 모델 튜닝 및 리더보드와 같은 기능을 활성화하고자 합니다.

🟢 평소와 같이 훈련 코드가 메트릭을 프린트하도록 하고, [use regular expressions](https://docs.aws.amazon.com/sagemaker/latest/dg/training-metrics.html#define-train-metrics)을 통해 세이지메이커가 작업 로그에서 구조화된 메트릭을 스크랩하는 방법을 정의하겠습니다:

In [ ]:
metric_definitions = [
    {"Name": "Epoch", "Regex": r"'epoch': ([0-9\.\-e]+)"},
    {"Name": "Train:Loss", "Regex": r"'loss': ([0-9\.\-e]+)"},
    {"Name": "Train:LearningRate", "Regex": r"'learning_rate': ([0-9\.\-e]+)"},
    {"Name": "Validation:Loss", "Regex": r"'eval_loss': ([0-9\.\-e]+)"},
    {"Name": "Validation:Accuracy", "Regex": r"'eval_accuracy': ([0-9\.\-e]+)"},
    {"Name": "Validation:F1", "Regex": r"'eval_f1': ([0-9\.\-e]+)"},
    {"Name": "Validation:Precision", "Regex": r"'eval_precision': ([0-9\.\-e]+)"},
    {"Name": "Validation:Recall", "Regex": r"'eval_recall': ([0-9\.\-e]+)"},
    {"Name": "Validation:Runtime", "Regex": r"'eval_runtime': ([0-9\.\-e]+)"},
    {"Name": "Validation:SamplesPerSecond", "Regex": r"'eval_samples_per_second': ([0-9\.\-e]+)"},
    {"Name": "Validation:StepsPerSecond", "Regex": r"'eval_steps_per_second': ([0-9\.\-e]+)"},
]
metric_definitions

## 세이지메이커에서 모델 훈련 및 검증하기

이번에는 노트북 자체와 별도의 인스턴스에서 훈련 프로세스를 실행하기 위해 [SageMaker training job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-training.html)을 생성하겠습니다: 이를 통해 수명이 긴 노트북 환경과 독립적으로 임시 훈련 인프라의 크기를 적절히 조정할 수 있습니다.

🟢 노트북에서 실제 훈련 코드를 **[scripts/train.py](scripts/train.py)**에 팩터링했으며, 이 스크립트에서 모델을 훈련하고 배포하기 위해 미리 빌드된 [세이지메이커 파이썬 SDK를 통한 허깅 페이스 프레임워크 컨테이너](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/index.html)를 사용할 것입니다.

### Amazon SageMaker가 사전 빌드된 컨테이너로 스크립트를 실행시키는 방법 

AWS는 주요 머신러닝 프레임워크에서 프로젝트를 빠르게 구축할 수 있도록 사전 패키지된 Docker 이미지 세트를 제공합니다:[SageMaker Framework Containers](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers-prebuilt.html).

이 컨테이너는 GPU 드라이버, 서빙 스택 구현, 핵심 라이브러리 등과 같은 기본 설정을 처리하므로 훈련 프로세스 및 추론 동작 오버라이드를 위한 Python 스크립트만 간단히 삽입하면 됩니다. 심지어 컨테이너 이미지에 빌드할 필요 없이 시작 시 동적으로 설치할 추가 종속성을 지정하는 *requirements.txt* 파일을 제공할 수도 있습니다.

**따라서 첫 번째 작업은 스크립트와 런타임 간의 인터페이스**를 이해하는 것입니다: 스크립트가 입력 데이터를 어떻게 읽을 것인가? 매개변수는? 결과를 어디에 저장해야 할까요?

#### 훈련 중인 컨테이너

훈련 작업 컨테이너가 시작되면 **코드 및 인풋 데이터**가 `/opt/ml` 디렉터리 아래의 **로컬 파일**로 다운로드됩니다. 또한 아래와 같이 학습된 모델과 기타 파일 출력을 로컬 파일 시스템에 저장합니다:


```
    /opt/ml
    |-- code
    |   `-- <our script(s)>
    |-- input
    |   |-- config
    |   |   |-- hyperparameters.json
    |   |   `-- resourceConfig.json
    |   `-- data
    |       `-- <channel_name>
    |           `-- <input data>
    |-- model
    |   `-- <model files>
    `-- output
        `-- failure
```


##### 인풋

* `/opt/ml/input/config`에는 프로그램 실행 방식을 제어하는 정보가 들어 있습니다. `hyperparameters.json`은 하이퍼파라미터 이름을 값으로 변환한 JSON 형식의 딕셔너리입니다. 이 값은 항상 문자열이므로 변환해야 할 수도 있습니다. `resourceConfig.json`은 분산 학습에 사용되는 네트워크 레이아웃을 설명하는 JSON 형식의 파일입니다. scikit-learn은 분산 학습을 지원하지 않으므로 여기서는 무시하겠습니다.
* (파일 모드의 경우) `/opt/ml/input/data/<channel_name>/`에는 해당 채널의 인풋 데이터가 포함됩니다. 채널은 CreateTrainingJob 호출에 따라 생성되지만 일반적으로 채널이 알고리즘이 예상하는 것과 일치하는 것이 중요합니다. 각 채널의 파일은 S3 키 구조로 표시된 트리 구조를 유지하면서 S3에서 이 디렉토리로 복사됩니다. 
* (파이프 모드의 경우) `/opt/ml/input/data/<channel_name>_<epoch_number>`는 주어진 에포크에 대한 파이프입니다. 에포크는 0에서 시작하여 읽을 때마다 하나씩 올라갑니다. 실행할 수 있는 에포크 수에는 제한이 없지만 다음 에포크를 읽기 전에 각 파이프를 닫아야 합니다.


##### 아웃풋

* `/opt/ml/model/`은 알고리즘이 생성하는 모델을 작성하는 디렉토리입니다. 모델은 원하는 모든 형식이 가능합니다. 단일 파일 또는 전체 디렉토리 트리가 될 수 있습니다. 세이지메이커는 이 디렉토리에 있는 모든 파일을 압축된 타르 아카이브 파일로 패키징합니다. 이 파일은 `DescribeTrainingJob` 결과에서 반환된 S3 위치에서 사용할 수 있습니다.
* `/opt/ml/output`은 알고리즘이 작업이 실패한 이유를 설명하는 `failure` 파일을 작성할 수 있는 디렉터리입니다. 이 파일의 내용은 `DescribeTrainingJob` 결과의 `FailureReason` 필드에 반환됩니다. 성공한 작업의 경우 무시되므로 이 파일을 작성할 이유가 없습니다.

#### 추가 정보

자세한 내용은 다음을 참조하세요:

- [SageMaker Python SDK guide for Hugging Face](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/index.html) 및 HF 프레임워크 클래스에 대한 [API doc](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html)를 참조하세요. (PyTorch의 해당 페이지도 유용할 수 있습니다).
- 기본 컨테이너 이미지를 정의하는 GitHub의 [AWS Deep Learning Containers repository](https://github.com/aws/deep-learning-containers).
- 훈련 및 서비스를 위한 프레임워크 코드에 대한 자세한 내용은 오픈 소스 [SageMaker Training Toolkit](https://github.com/aws/sagemaker-training-toolkit) 및 [SageMaker Inference Toolkit](https://github.com/aws/sagemaker-inference-toolkit)을 참조하세요. (일부 프레임워크는 이러한 툴킷의 변형을 사용합니다. 예: [sagemaker-pytorch-training-toolkit](https://github.com/aws/sagemaker-pytorch-training-toolkit)).

### 작업 생성하기

실제 [SageMaker CreateTrainingJob API](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateTrainingJob.html)에는 세이지메이커 파이썬 SDK의 상위 [high-level 'Estimator' classes](https://sagemaker.readthedocs.io/en/stable/overview.html)가 단순화하는 데 도움이 되는 몇 가지 하위 레벨 세부 정보가 필요합니다. 특히:

- 정확한 컨테이너 이미지 URI를 지정하는 대신 선택한 프레임워크 및 버전에 따라 SDK가 이를 찾아줍니다.
- SDK는 `scripts` 번들을 투명하게 압축하여 S3에 업로드하고, 거기에서 로드하도록 훈련 작업을 구성합니다.

먼저, 작업을 구성하는 `estimator` 객체와 이 작업이 실행될 인프라(컴퓨팅 인스턴스 수와 유형)를 생성합니다:

> ℹ️ 사용자를 대신하여 작업을 실행하는 다른 서비스와 마찬가지로, 훈련 작업은 [IAM 역할](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)을 설정하여 S3의 인풋 훈련 데이터와 같은 리소스에 액세스할 수 있도록 합니다. 세이지메이커 노트북 자체는 이미 가정된 역할로 실행되므로, 단순화를 위해 훈련 작업 역할을 노트북 역할과 동일하게 설정하겠습니다.

In [ ]:
from sagemaker.huggingface.estimator import HuggingFace as HuggingFaceEstimator

nb_role = sagemaker.get_execution_role()

estimator = HuggingFaceEstimator(
    transformers_version="4.26",
    pytorch_version="1.13",
    py_version="py39",

    source_dir="scripts",  # Local folder where fine-tuning script is stored
    entry_point="train.py",  # Actual script the training job should run

    base_job_name="news-classifier",  # Prefix for the training job name (timestamp will be added)
    instance_count=1,  # Number of instances train on (need to prepare your script for using >1!)
    instance_type="ml.p3.2xlarge",  # Type of compute instance to use: p* and g* include GPUs
    role=nb_role,  # IAM role the job will use to access AWS resources (e.g. data on S3)

    hyperparameters= hyperparameters,   # Training job parameters, as we set up earlier
    metric_definitions=metric_definitions,  # RegEx to extract metric data from training job logs
)

구성이 완료되면 `estimator.fit()`을 실행하고 인풋 데이터 위치를 지정하여 실제 훈련 작업을 시작할 수 있습니다.

데이터 인풋 '채널'의 수, 이름 및 유형은 [사용자에게 달려 있습니다](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateTrainingJob.html#sagemaker-CreateTrainingJob-request-InputDataConfig): 노트북이 스크립트에서 예상하는 것과 동일한 채널을 구성하는지 확인하세요.

In [ ]:
%%time

estimator.fit(
    {
        "train": train_s3_uri,
        "test": test_s3_uri,
    },
    wait=True,  # Wait for the training to complete (default=True)
    logs=True,  # Stream training job logs to the notebook (default=True, requires wait=True)
)

> ⏰ 이 훈련 작업은 완료하는 데 10분 정도 걸리지만 '로컬' 모델보다 훨씬 더 높은 정확도에 도달해야 합니다.

트레이닝 자체는 소형 CPU 전용 노트북이 아닌 GPU 가속 인스턴스에서 실행되므로 이전의 '로컬' 예제보다 훨씬 빨라야 합니다. 하지만 인프라를 프로비저닝하고 작업을 시작하는 데는 몇 분 정도 소요될 수 있습니다.

[아마존 세이지메이커용 AWS Console](https://console.aws.amazon.com/sagemaker/home?#/jobs)의 *Training jobs* 페이지와 세이지메이커 스튜디오의 **Experiments** UI(왼쪽 사이드바의 🏠 **홈** 버튼에서)에서도 현재 및 과거 작업의 상태를 확인할 수 있습니다.

🟢 대기 및 로그 스트리밍의 기본 동작은 로컬과 유사한 경험을 제공하지만, 훈련 작업은 노트북에 의존하지 않습니다:

- 노트북의 연결을 끊거나 종료해도 훈련 작업은 계속 진행됩니다.
- `wait=False`를 설정하면 노트북에서 여러 개의 트레이닝 작업을 동시에 시작할 수 있습니다.
- 재시작한 노트북을 이전 훈련 작업에 연결해야 하는 경우, 아래와 같이 훈련 작업 이름으로 `.attach()`하면 됩니다:

In [ ]:
# estimator = HuggingFaceEstimator.attach("news-classifier-2023-03-30-16-09-35-281")

훈련 작업이 완료되면 컨테이너의 모델 아웃풋 폴더의 내용이 자동으로 S3에 아카이브됩니다.

아래 그림과 같이 이 파일을 참조할 수 있으며, SageMaker 외부에서 학습된 모델을 유사한 타르볼 형식으로 준비하여 배포용으로 가져올 수도 있습니다:

In [ ]:
estimator.latest_training_job.describe()["ModelArtifacts"]["S3ModelArtifacts"]

## 추론에 모델 사용

모델이 학습되면 새로운 데이터에 대한 추론에 사용할 준비가 된 것입니다.

세이지메이커는 [온디맨드 추론을 위한 모델 배포](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html) 또는 [배치 추론 작업 실행](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html)을 위한 여러 가지 완전 관리형 옵션을 제공합니다.

> ℹ️ **주의사항:** 사용 사례에 적합한 추론 옵션을 선택하세요 - 배치 데이터만 처리하려는 경우 실시간 엔드포인트를 배포할 필요가 없습니다!
>
> 지금까지 사용했던 것과 동일한 높은 수준의 SageMaker Python SDK를 통해 배치 추론을 실행하는 방법에 대한 자세한 내용은 [SageMaker 배치 변환 사용](https://sagemaker.readthedocs.io/en/stable/overview.html#sagemaker-batch-transform)을 참조하십시오.

이 예제에서는 모델을 [실시간 추론 엔드포인트](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html)에 배포하여 온디맨드 방식으로 헤드라인을 분류할 수 있도록 하겠습니다.

엔드포인트를 실행할 인프라 유형을 다시 지정하므로 시작하는 데 몇 분 정도 걸립니다. 이 테스트 엔드포인트는 트래픽을 매우 적게 처리하므로 더 작고 저렴한 인프라를 사용할 수 있으므로 훈련용과는 다른 유형의 인스턴스를 사용할 수 있습니다:

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
)

배포 후에는 [Amazon SageMaker용 AWS 콘솔](https://console.aws.amazon.com/sagemaker/home?#/endpoints)의 *Endpoints* 페이지와 세이지메이커 스튜디오 UI의 **Deployments > Endpoints** 섹션(왼쪽 사이드바의 🏠 **Home** 버튼에서)에서 엔드포인트를 찾을 수 있어야 합니다.

훈련 작업과 마찬가지로 엔드포인트는 노트북 자체에서 분리되어 있습니다. 다음과 같이 이전에 배포된 엔드포인트에 노트북을 연결할 수 있습니다:

In [ ]:
# from sagemaker.huggingface import HuggingFacePredictor
# predictor = HuggingFacePredictor("news-classifier-2023-03-24-13-31-09-895")

### 이제 모델이 프로덕션 환경에서 RESTful API로 작동합니다!

[Predictor](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html)는 여기서 모델을 메모리에 로드하지 않고 대신 배포된 엔드포인트에 대한 HTTPS API 호출을 래핑합니다.

여기서는 Hugging Face 프레임워크에서 제공하는 기본 `application/json` 직렬화 지원을 사용하고 있지만, 프레임워크마다 기본 형식이 다르므로 사용자 정의 [serializers](https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html) 및 [deserializers](https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html)(client/`predictor` 측)와 사용자 정의 [`input_fn`s and `output_fn`s](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#process-model-input)(엔드포인트 컨테이너 측)를 사용하여 원하는 거의 모든 요청 또는 응답 형식을 설정할 수 있습니다: 자체 서빙 스택을 처음부터 작성할 필요가 없습니다.

요청 역/직렬화 및 처리는 이미 [HuggingFacePredictor](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#hugging-face-predictor)와 사전 구축된 추론 컨테이너에 의해 처리되므로, 노트북에서 배포된 모델을 호출하는 것은 로컬 인메모리 모델을 호출하는 것만큼이나 쉽습니다:

In [ ]:
def classify(text: str) -> dict:
    """Classify a headline and print the results"""
    return predictor.predict({"inputs":[text]})[0]


# Either try out the interactive widget:
interaction = widgets.interact_manual(
    classify,
    text=widgets.Text(
        value="The markets were bullish after news of the merger",
        placeholder="Type a news headline...",
        description="Headline:",
        layout=widgets.Layout(width="99%"),
    ),
)
interaction.widget.children[1].description = "Classify!"

또는 (예를 들어 UI 위젯 라이브러리로 어려움을 겪고 있는 경우) 코드에서 직접 엔드포인트를 호출할 수 있습니다:

In [ ]:
classify("Retailers are expanding after the recent economic growth")

## 정리

세이지메이커 작업(예: 훈련, 처리 및 배치 추론)은 실행하는 동안에만 온디맨드 컴퓨팅을 사용하지만, 배포된 실시간 추론 엔드포인트는 해제할 때까지 리소스를 계속 소비한다는 점에 유의하세요.

실험이 끝나면 더 이상 필요하지 않은 엔드포인트를 삭제하여 불필요한 비용을 피하세요:

In [ ]:
# predictor.delete_endpoint(delete_endpoint_config=True)

## 리뷰

이 노트북에서는 Amazon SageMaker에서 허깅 페이스 트랜스포머를 사용해 텍스트 분류 모델을 훈련하고 배포하는 방법을 보여드렸습니다.

이 접근법의 몇 가지 이점은 함께 제공되는 [Headline Classifier Local notebook](Headline%20Classifier%20Local.ipynb)과 비교했을 때 다음과 같습니다:

- **훈련 작업 기간 동안만** 전문 컴퓨팅 리소스(예: 고성능 또는 GPU 가속 인스턴스)를 자동으로 프로비저닝할 수 있습니다: 활용도가 낮은 리소스를 방치하지 않고 트레이닝에서 우수한 성능을 얻을 수 있습니다.
- 사용자가 무엇이 효과가 있고 무엇이 효과가 없었는지를 기록해야 하는 로컬 노트북 실험과 달리, 훈련 작업의 이력(매개변수, 메트릭, 아웃풋 등)이 자동으로 추적됩니다.
- 학습된 모델은 단 한 번의 SDK 호출로 프로덕션에 바로 사용할 수 있는 안전한 웹 엔드포인트에 배포할 수 있습니다: 동작을 심층적으로 사용자 정의하려는 경우가 아니라면 컨테이너나 웹 애플리케이션 패키징이 필요하지 않습니다.

로컬 노트북과 이 SageMaker 버전 및 함께 제공되는 [scripts/train.py](스크립트/트레인.py) 스크립트 파일을 비교하면 자체 또는 오픈 소스 노트북 내 ML 워크플로를 SageMaker "스크립트 모드" 훈련 작업으로 마이그레이션하는 방법에 대한 아이디어를 얻을 수 있습니다.

이 워크샵의 다음 '마이그레이션 챌린지' 연습에서는 다른 '로컬' 노트북에 대해 이 과정을 직접 반복해 보겠습니다.

또한 SageMaker 작업을 파이프라인으로 연결하고 CI/CD로 워크플로우를 자동화하는 등의 추가 단계를 보여주는 [aws-samples/amazon-sagemaker-from-idea-to-production](https://github.com/aws-samples/amazon-sagemaker-from-idea-to-production)에도 관심이 있을 경우 참고하세요.